## Measuring VNC length

To measure the VNC length, we need to go over these steps:

1. Classify embryo orientation
1. Calculate an ROI for each image slice, based on predicted orientation
2. Measure the maximum feret diameter of each slice

> The maximum feret diameter is computed as the longest distance between points around a region's convex hull contour.

The measurement is done using the structural channel, so that the pixel intensity changes less over time, making the ROI easier to calculate.
There is a lot of fluctuation in the lengths, because the embryo flickers over time, causing the VNC to change position and deform.
Part of the change in the VNC length is that the ROI sometimes changes and accounts for regions that are actually part of the brain lobes.

To filter out the effects of movement and deformation, a baseline with a first degree polynomial was plotted with the image.

Because we are using the ROI to calculate the VNC length, the more accurate our ROI is, the better the results will be.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os
from pathlib import Path

from tifffile import imread
import matplotlib.pyplot as plt

from pasnascope import roi, vnc_length, classifier, initial_mask, find_hatching, projs

experiment_name = '20240307'
root_dir = Path(os.getcwd()).parent
project_dir = os.path.join(root_dir, 'data', experiment_name)

In [ ]:
def get_vnc_length_ell_fit(file_path, hp, interval=20, pixel_width=1.62):
    '''Measures VNC length, first masks the image with an ellipse.'''
    img = imread(file_path, key=range(0,hp, interval))
    ell = initial_mask.fit_ellipse(img)
    mask = initial_mask.create_mask_from_ellipse(ell, img[0].shape)
    img_roi = roi.get_roi(img, window=1, mask=mask)
    return vnc_length.measure_VNC(img_roi)*pixel_width

def get_vnc_length_from_tiff(file_path, hp, interval=20, pixel_width=1.62):
    '''Measures VNC length using the `pasnascope` module.'''
    img = imread(file_path, key=range(0,hp, interval))
    img_roi = roi.get_roi(img, window=1)
    return vnc_length.measure_VNC(img_roi)*pixel_width

def avg(l):
    return sum(l)/len(l)


PIXEL_WIDTH = 1.62

img_dir = os.path.join(project_dir, 'embs')
model_path = os.path.join(project_dir, 'models', 'SVC')
imgs = [f for f in os.listdir(img_dir) if f.endswith('ch2.tif')]

lengths = []

for img in imgs:
    img_path = os.path.join(img_dir, img)
    orientation = classifier.classify_image(img_path, model_path)
    
    hp = find_hatching.find_hatching_point(img_path)

    if orientation == 'v':
        vncl = get_vnc_length_from_tiff(img_path, hp, interval=20)
    elif orientation == 'l':
        vncl = get_vnc_length_ell_fit(img_path, hp, interval=20)

    lengths.append(vncl)

deltas = [(avg(l[-10:]) - avg(l[:10]))/avg(l[:10]) for l in lengths]
x = [i for i, _ in enumerate(deltas)]

fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
ax.plot(x ,deltas)

In [ ]:
def get_vnc_length_projs(file_path, hp, interval=20, pixel_width=1.62):
    '''Measures VNC length, first masks the image with an ellipse.'''
    img = imread(file_path, key=range(0,hp, interval))
    mask = projs.proj_mask(img[0])
    img_roi = roi.get_roi(img, window=1, mask=mask)
    return vnc_length.measure_VNC(img_roi)*pixel_width

def avg(l):
    return sum(l)/len(l)


PIXEL_WIDTH = 1.62

img_dir = os.path.join(project_dir, 'embs')
model_path = os.path.join(project_dir, 'models', 'SVC')
imgs = [f for f in os.listdir(img_dir) if f.endswith('ch2.tif')][:8]

lengths = []

for img in imgs:
    img_path = os.path.join(img_dir, img)
    
    hp = find_hatching.find_hatching_point(img_path)
    vncl = get_vnc_length_projs(img_path, hp, 20)
    lengths.append(vncl)

fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
for l in lengths:
    ax.plot(l)

Next cell compares the VNC length calculation for the worst embryo configuration of experiment 20240220.

Using `roi.get_roi()` without a mask results in measurements very different than the annotated data.
When we add the projection mask, the VNC length is a bit higher, but the same pattern remains, meaning that the errors caused by adding the brain lobes region to the ROI are eliminated (at least parcially).

In [ ]:
def get_vnc_length_raw(file_path, hp, interval=20, pixel_width=1.62):
    '''Measures VNC length, without masking.'''
    img = imread(file_path, key=range(0,hp, interval))
    img_roi = roi.get_roi(img, window=1, mask=None)
    return vnc_length.measure_VNC(img_roi)*pixel_width

def get_vnc_length_projs(file_path, hp, interval=20, pixel_width=1.62):
    '''Measures VNC length, first masks the image with an ellipse.'''
    img = imread(file_path, key=range(0,hp, interval))
    mask = projs.proj_mask(img[0])
    img_roi = roi.get_roi(img, window=1, mask=mask)
    return vnc_length.measure_VNC(img_roi)*pixel_width

def avg(l):
    return sum(l)/len(l)


PIXEL_WIDTH = 1.62

img_dir = os.path.join(project_dir, 'embs')
imgs = [f for f in os.listdir(img_dir) if f.endswith('ch2.tif')][:1]
print(imgs)

lengths = []
lengths_raw = []

for img in imgs:
    img_path = os.path.join(img_dir, img)
    
    hp = find_hatching.find_hatching_point(img_path)
    vnclr = get_vnc_length_raw(img_path, hp, 20)
    vncl = get_vnc_length_projs(img_path, hp, 20)

    lengths.append(vncl)
    lengths_raw.append(vnclr)

csv_path = os.path.join(project_dir, 'annotated', 'emb33-ch2-lengths.csv' )
manual = vnc_length.get_length_from_csv(csv_path)
print(manual.shape)
x = manual[:, 0]
y = manual[:, 1]


fig, ax = plt.subplots(2)
fig.canvas.header_visible = False
fig.canvas.resizable = False

ax[0].plot(x, lengths[0][:-2], color='r')
ax[0].plot(x, y, label='manual', color='g')
ax[0].set_title('With proj mask')
ax[1].plot(x, lengths_raw[0][:-2], color='b')
ax[1].plot(x, y, label='manual', color='g')
ax[1].set_title('Without mask')

plt.tight_layout()

In [ ]:
def get_length_centerline(file_path, hp, interval=20, pixel_width=1.62):
    '''Measures VNC length, without masking.'''
    img = imread(file_path, key=range(0,hp, interval))
    return vnc_length.measure_VNC_centerline(img)*pixel_width

interval = 20
img_dir = os.path.join(project_dir, 'embs')
annotated_dir = os.path.join(project_dir, 'annotated')
embs = [f.split('.')[0] for f in os.listdir(annotated_dir)][:3]

lengths = []
ann_lengths = []

for emb in embs:
    img_path = os.path.join(img_dir, f"{emb}.tif")
    hp = find_hatching.find_hatching_point(img_path)
    # adjust hatching point to make it comparable with annotated data
    hp -= hp % interval

    vnc_len = get_length_centerline(img_path, hp, interval=interval)
    lengths.append(vnc_len)

    csv_path = os.path.join(annotated_dir, f"{emb}.csv")
    csv_end = hp//interval
    manual = vnc_length.get_length_from_csv(csv_path, csv_end)
    ann_lengths.append(manual)

fig, ax = plt.subplots(len(embs))
ax = ax.ravel()
fig.canvas.header_visible = False
fig.canvas.resizable = False
fig.suptitle('Centerline estimation')
for i, (l, al) in enumerate(zip(lengths, ann_lengths)):
    x = list(range(0, len(l)*interval, interval))
    ax[i].plot(x, l, color='r', label='calculated')
    ax[i].plot(x, al, color='g', label='annotated')
ax[0].legend()

plt.tight_layout()

### Effect of sampling when measuring VNC length

Different intervals where selected to check how they affect the change in the VNC length.
The idea is to also see if the measurements taken manually can be directly compared with the values in here. 

In [ ]:
img_dir = os.path.join(project_dir, 'embs')
imgs = [f for f in os.listdir(img_dir) if f.endswith('ch2.tif')]

i = 0
PIXEL_WIDTH = 1.62
img = imread(os.path.join(img_dir , imgs[i]), key=range(0,1000))

img_roi = roi.get_roi(img, window=1)
vnc_lengths = vnc_length.measure_VNC(img_roi)*PIXEL_WIDTH

reg = vnc_length.fit_regression(vnc_lengths)

windows = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
deltas = []
for w in windows:
    vnc_samples = vnc_lengths[::w]
    reg = vnc_length.fit_regression(vnc_samples)
    deltas.append((reg(vnc_samples.size-1)-reg(0))/reg(0))

fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
fig.suptitle("% condensation x sampling interval")
ax.plot(windows, deltas, 'r:o');